In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import json
import pandas_profiling

In [3]:
# in case you don't have file downloaded yet
# or run on google colab 
gist_path = "https://gist.githubusercontent.com/ttpro1995/391c8df70dd780a64a544cce8d40de03/raw/c2374358d7a6a2afd88826dcd8dbb673918f42ae/train.jsonl"

In [4]:
df = pd.read_json(gist_path, lines=True)

In [5]:
df.head()

,train_id,original_doc,html_annotation,modified_time,status,match_summary
0,21223385,"{'_id': '21223385', '_source': {'body': [{'con...","[<span>Video tổng hợp trận đấu:</span>, <span ...",2020-10-15 11:37:37.208116,accept,"{'players': {'team1': 'Chelsea', 'team2': 'Sto..."
1,21231305,"{'_id': '21231305', '_source': {'body': [{'con...","[<span class=""tag match_info"" data="":match_inf...",2020-10-19 21:54:18.727348,accept,"{'players': {'team1': 'Arsenal', 'team2': 'Cry..."
2,21231956,"{'_id': '21231956', '_source': {'body': [{'con...","[<span class=""tag match_info"" data="":match_inf...",2020-10-19 22:40:33.078740,accept,"{'players': {'team1': 'Chelsea', 'team2': 'Sto..."
3,21231957,"{'_id': '21231957', '_source': {'body': [{'con...","[<span class=""tag match_info"" data="":match_inf...",2020-10-21 08:57:08.463983,accept,"{'players': {'team1': 'Watford', 'team2': 'Tot..."
4,21231959,"{'_id': '21231959', '_source': {'body': [{'con...","[<span class=""tag match_info"" data="":match_inf...",2020-09-18 22:25:17.596924,accept,"{'players': {'team1': 'Arsenal', 'team2': 'Cry..."


# Đọc html và chuyển sang bài toán phân loại event 

## Chuyển sang list của các dict 

In [7]:
df1 = df.drop(["original_doc", "modified_time", "status", "match_summary"], axis=1)

In [8]:
df1.head()

,train_id,html_annotation
0,21223385,"[<span>Video tổng hợp trận đấu:</span>, <span ..."
1,21231305,"[<span class=""tag match_info"" data="":match_inf..."
2,21231956,"[<span class=""tag match_info"" data="":match_inf..."
3,21231957,"[<span class=""tag match_info"" data="":match_inf..."
4,21231959,"[<span class=""tag match_info"" data="":match_inf..."


In [10]:
doc = df1.iloc[0]
doc

train_id                                                    21223385
html_annotation    [<span>Video tổng hợp trận đấu:</span>, <span ...
Name: 0, dtype: object

In [18]:
def html_annotation_to_list(doc):
    event_list = []
    for html in doc["html_annotation"]:
        soup = BeautifulSoup(html)
        events = soup.find_all("span", {"class": "tag"})
        for e in events:
            item_dict = dict()
            item_dict["event_type"] = e["data"]
            item_dict["event_id"] = e["event_id"]
            item_dict["text"] = e.text
            event_list.append(item_dict)
    return event_list

In [17]:
event_list

[{'event_type': ':match_info:match_result:',
  'event_id': '1',
  'text': 'Chelsea cần chiến thắng trước Stoke City ở Stamford Bridge để cân bằng kỷ lục 13 chiến thắng của Arsenal ở cuối mùa 2001/02. Cuối cùng, đoàn quân của HLV Conte đã hoàn thành nhiệm vụ này khi vượt qua đối thủ với tỷ số 4-2'},
 {'event_type': ':goal_info:',
  'event_id': '2',
  'text': 'Tuy vậy, 1 phút sau, Chelsea đã cụ thể hóa sức ép liên tục bằng bàn thắng. Người lập công cho The Blues là đội trưởng Cahill với cú đánh đầu cận thành từ tình huống phạt góc.'},
 {'event_type': ':goal_info:',
  'event_id': '8',
  'text': 'Ngay đầu hiệp 2, Stoke City đã bất ngờ tìm được bàn thắng gỡ hòa. Từ tình huống cố định, Crouch đã nhả đầu cho Indi đệm bóng vào lưới thủ thành Courtois.'},
 {'event_type': ':goal_info:',
  'event_id': '3',
  'text': 'Phút 57, sau pha nhả bóng của Hazard, Willian đã tung ra cú dứt điểm cháy lưới của Stoke City, nâng tỷ số lên 2-1.'},
 {'event_type': ':goal_info:',
  'event_id': '4',
  'text': 'Kịc

In [20]:
df1["list_annotation"] = df1.apply(html_annotation_to_list, axis=1)

In [21]:
df1.head()

,train_id,html_annotation,list_annotation
0,21223385,"[<span>Video tổng hợp trận đấu:</span>, <span ...","[{'event_type': ':match_info:match_result:', '..."
1,21231305,"[<span class=""tag match_info"" data="":match_inf...","[{'event_type': ':match_info:', 'event_id': '1..."
2,21231956,"[<span class=""tag match_info"" data="":match_inf...","[{'event_type': ':match_info:', 'event_id': '1..."
3,21231957,"[<span class=""tag match_info"" data="":match_inf...","[{'event_type': ':match_info:', 'event_id': '1..."
4,21231959,"[<span class=""tag match_info"" data="":match_inf...","[{'event_type': ':match_info:', 'event_id': '5..."


## bung array ra các row 

In [22]:
df2 = df1.explode("list_annotation")

In [23]:
df2.head()

,train_id,html_annotation,list_annotation
0,21223385,"[<span>Video tổng hợp trận đấu:</span>, <span ...","{'event_type': ':match_info:match_result:', 'e..."
0,21223385,"[<span>Video tổng hợp trận đấu:</span>, <span ...","{'event_type': ':goal_info:', 'event_id': '2',..."
0,21223385,"[<span>Video tổng hợp trận đấu:</span>, <span ...","{'event_type': ':goal_info:', 'event_id': '8',..."
0,21223385,"[<span>Video tổng hợp trận đấu:</span>, <span ...","{'event_type': ':goal_info:', 'event_id': '3',..."
0,21223385,"[<span>Video tổng hợp trận đấu:</span>, <span ...","{'event_type': ':goal_info:', 'event_id': '4',..."


## Chuyển dict thành các cột text, event_type, event_id

In [27]:
df2["event_type"] = df2["list_annotation"].apply(lambda doc: doc["event_type"])
df2["event_id"] = df2["list_annotation"].apply(lambda doc: doc["event_id"])
df2["text"] = df2["list_annotation"].apply(lambda doc: doc["text"])

In [28]:
df2.head()

,train_id,html_annotation,list_annotation,event_type,event_id,text
0,21223385,"[<span>Video tổng hợp trận đấu:</span>, <span ...","{'event_type': ':match_info:match_result:', 'e...",:match_info:match_result:,1,Chelsea cần chiến thắng trước Stoke City ở Sta...
0,21223385,"[<span>Video tổng hợp trận đấu:</span>, <span ...","{'event_type': ':goal_info:', 'event_id': '2',...",:goal_info:,2,"Tuy vậy, 1 phút sau, Chelsea đã cụ thể hóa sức..."
0,21223385,"[<span>Video tổng hợp trận đấu:</span>, <span ...","{'event_type': ':goal_info:', 'event_id': '8',...",:goal_info:,8,"Ngay đầu hiệp 2, Stoke City đã bất ngờ tìm đượ..."
0,21223385,"[<span>Video tổng hợp trận đấu:</span>, <span ...","{'event_type': ':goal_info:', 'event_id': '3',...",:goal_info:,3,"Phút 57, sau pha nhả bóng của Hazard, Willian ..."
0,21223385,"[<span>Video tổng hợp trận đấu:</span>, <span ...","{'event_type': ':goal_info:', 'event_id': '4',...",:goal_info:,4,Kịch tính của trận đấu được đẩy lên khi Stoke ...


## Bỏ bớt cột không cần thiết 

In [32]:
df3 = df2.drop(["html_annotation", "list_annotation"], axis=1)

In [60]:
df3.head()

,train_id,event_type,event_id,text,goal_info,match_info
0,21223385,:match_info:match_result:,1,Chelsea cần chiến thắng trước Stoke City ở Sta...,0,1
0,21223385,:goal_info:,2,"Tuy vậy, 1 phút sau, Chelsea đã cụ thể hóa sức...",1,0
0,21223385,:goal_info:,8,"Ngay đầu hiệp 2, Stoke City đã bất ngờ tìm đượ...",1,0
0,21223385,:goal_info:,3,"Phút 57, sau pha nhả bóng của Hazard, Willian ...",1,0
0,21223385,:goal_info:,4,Kịch tính của trận đấu được đẩy lên khi Stoke ...,1,0


Bây giờ, ta xem đây là bài toán phân loại. Ta sẽ huấn luyện dữ liệu chữ (cột `text`) để dự đoán event type của cột đó. Event type có 5 loại - `goal_info`, `match_info`, `match_result`, `card_info`, `substitution` và mỗi text có thể thuộc nhiều event type.

## Tạo thêm cột cho mỗi event type 
Đó là các cột `goal_info`, `match_info`, `match_result`, `card_info`, `substitution` 

Ta thêm các cột đó để huấn luyện nhiều 1 vs many classifier. Mỗi classifier sẽ đánh giá xem text có thuộc event type đó không.

In [63]:
def get_goal_info(doc):
    event = "goal_info"
    if event in doc["event_type"]:
        return 1
    else:
        return 

In [64]:
def get_match_info(doc):
    event = "match_info"
    if event in doc["event_type"]:
        return 1
    else:
        return 0

In [65]:
def get_match_result(doc):
    event = "match_result"
    if event in doc["event_type"]:
        return 1
    else:
        return 0

In [66]:
def get_card_info(doc):
    event = "card_info"
    if event in doc["event_type"]:
        return 1
    else:
        return 0

In [67]:
def get_substitution(doc):
    event = "substitution"
    if event in doc["event_type"]:
        return 1
    else:
        return 0

In [68]:
df3["goal_info"] = df3.apply(get_goal_info, axis=1)

In [69]:
df3["match_info"] = df3.apply(get_match_info, axis=1)

In [70]:
df3["match_result"] = df3.apply(get_match_result, axis=1)

In [71]:
df3["card_info"] = df3.apply(get_card_info, axis=1)

In [72]:
df3["substitution"] = df3.apply(get_substitution, axis=1)

In [73]:
df3.head()

,train_id,event_type,event_id,text,goal_info,match_info,match_result,card_info,substitution
0,21223385,:match_info:match_result:,1,Chelsea cần chiến thắng trước Stoke City ở Sta...,0,1,1,0,0
0,21223385,:goal_info:,2,"Tuy vậy, 1 phút sau, Chelsea đã cụ thể hóa sức...",1,0,0,0,0
0,21223385,:goal_info:,8,"Ngay đầu hiệp 2, Stoke City đã bất ngờ tìm đượ...",1,0,0,0,0
0,21223385,:goal_info:,3,"Phút 57, sau pha nhả bóng của Hazard, Willian ...",1,0,0,0,0
0,21223385,:goal_info:,4,Kịch tính của trận đấu được đẩy lên khi Stoke ...,1,0,0,0,0


# Xong tạo data cho bài toán phân loại 1 vs all 

In [75]:
df3.head() 

,train_id,event_type,event_id,text,goal_info,match_info,match_result,card_info,substitution
0,21223385,:match_info:match_result:,1,Chelsea cần chiến thắng trước Stoke City ở Sta...,0,1,1,0,0
0,21223385,:goal_info:,2,"Tuy vậy, 1 phút sau, Chelsea đã cụ thể hóa sức...",1,0,0,0,0
0,21223385,:goal_info:,8,"Ngay đầu hiệp 2, Stoke City đã bất ngờ tìm đượ...",1,0,0,0,0
0,21223385,:goal_info:,3,"Phút 57, sau pha nhả bóng của Hazard, Willian ...",1,0,0,0,0
0,21223385,:goal_info:,4,Kịch tính của trận đấu được đẩy lên khi Stoke ...,1,0,0,0,0


Label là các cột `goal_info`, `match_info`, `match_result`, `card_info`, `substitution`

Lưu ra .csv

In [78]:
df3.to_csv("data/event_classifier_1_vs_many.csv", index=False)